In [1]:
import numpy as np
import pandas as pd

In [2]:
# epl_data 자체를 가공하는 유틸리티 클라스를 정의.
class DataHandler:
    def __init__(self):
        pass
    
    def load_epl(self):
        # read data
        df = pd.read_csv('results.csv', encoding='cp949')
        
        # NaN 값이 들어있는 행을 모두 제거(2000-01 시즌부터 데이터를 활용), 인덱스를 리셋.
        df.dropna(axis=0, inplace=True)
        df.reset_index(inplace=True, drop=True)
        
        # 분석에 필요없는 정보는 삭제
        df.drop(['Referee'], axis=1, inplace=True)
        
        # epl_data에서 object 형식인 column들을 홈팀의 승점으로 변환시켜준다.
        df['FTR'] = df['FTR'].replace(['H', 'D', 'A'],[3, 1, 0])    
        df['HTR'] = df['HTR'].replace(['H', 'D', 'A'],[3, 1, 0])
        return df

    def season_split(self, df, index_reset=False):# index_reset을 True로 놓으면 시즌 데이터의 index도 0부터 시작.
        # 시즌별로 데이터를 나눈다.
        season_data_list = []

        seasons = []
        for i in range(22):
            if i <= 8:
                former = '0'+str(i); post = '0'+str(i+1)
            elif i == 9:
                former = '09'; post = '10'
            else:
                former = str(i); post = str(i+1)
            season = '20'+former+'-'+post
            seasons.append(season)
        
        for season in seasons:
            epl_season_data = df[df['Season']==season]
            if index_reset == True:
                epl_season_data.reset_index(inplace=True, drop=True)
            season_data_list.append(epl_season_data)
        return season_data_list
    
    # 원본 데이터에서 해당 팀이 치룬 모든 경기를 새로운 DataFrame으로 반환해준다.
    def all_team_matches(self, df, team):
        last_games_home = df[df['HomeTeam'] == team]
        last_games_away = df[df['AwayTeam'] == team]
        
        last_games = pd.concat([last_games_home, last_games_away])
        last_games.sort_values('DateTime', ascending=True, inplace=True)
        last_games.reset_index(drop=True, inplace=True)
        return last_games
    
    # 원본 데이터에서 입력한 팀의 홈에서의 경기들을 새로운 DataFrame으로 반환해준다.
    def home_matches(self, df, team):
        matches = df[df['HomeTeam'] == team]
        matches.reset_index(drop=True, inplace=True)
        return matches
    
    # 원본 데이터에서 입력한 팀의 원정에서의 경기들을 새로운 DataFrame으로 반환해준다.
    def away_matches(self, df, team):
        matches = df[df['AwayTeam'] == team]
        matches.reset_index(drop=True, inplace=True)
        return matches
    
    # 원본 데이터에서 입력한 두 팀 간의 경기들을 새로운 DataFrame으로 반환해준다. home_setting이 team1의 홈에서의 경기 결과들을 반환한다.
    def all_two_team_matches(self, df, team1, team2, home_setting=False):
        cond1 = (df['HomeTeam']==team1) & (df['AwayTeam']==team2)
        cond2 = (df['HomeTeam']==team2) & (df['AwayTeam']==team1)
        if home_setting == True:
            team1_home_matches = df[cond1]
            team1_home_matches.reset_index(drop=True, inplace=True)
            return team1_home_matches
        else:
            all_matches = df[cond1 | cond2]
            all_matches.reset_index(drop=True, inplace=True)
            return all_matches

In [3]:
dhd = DataHandler()
epl_data = dhd.load_epl()

In [4]:
# data_transform 안에서 필요한 유틸리티 클라스를 하나 정의.
class DataTransformer:
    # 클래스 변수로 DataHandler 객체를 놓는다.
    dhd = DataHandler()
    
    def __init__(self):
        pass
    
    # 경기 결과들이 들어간 DataFrame을 넣으면 시즌 정보(현재까지의 경기 정보) DataFrame을 반환.
    def season_result(self, season_data):    
        teams = set(season_data['HomeTeam'].values.tolist()) | set(season_data['AwayTeam'].values.tolist())

        team_datas = []
        for team in teams:
            Win = 0; Draw = 0; Lose = 0; Pts = 0; Score = 0; Loss = 0; GD = 0

            # 홈 경기 결과 정리
            home_matches = season_data[season_data['HomeTeam']==team]

            Win += home_matches[home_matches['FTR']==3].shape[0]
            Draw += home_matches[home_matches['FTR']==1].shape[0]
            Lose += home_matches[home_matches['FTR']==0].shape[0]

            Score += home_matches['FTHG'].sum()
            Loss += home_matches['FTAG'].sum()

            # 어웨이 경기 결과 정리
            away_matches = season_data[season_data['AwayTeam']==team]

            Win += away_matches[away_matches['FTR']==0].shape[0]
            Draw += away_matches[away_matches['FTR']==1].shape[0]
            Lose += away_matches[away_matches['FTR']==3].shape[0]

            Score += away_matches['FTAG'].sum()
            Loss += away_matches['FTHG'].sum()

            # 승점, 골득실 정리
            Pts = Win*3 + Draw*1; GD = Score - Loss

            team_data = [team, Win, Draw, Lose, Pts, Score, Loss, GD]
            team_datas.append(team_data)

        col_name = ['Team', 'W', 'D', 'L', 'Pts', 'Score','Loss', 'GD']
        season_result = pd.DataFrame(team_datas, columns = col_name)
        season_result.sort_values(['Pts', 'GD', 'Score'], ascending=[False, False, False], inplace=True)
        season_result.reset_index(inplace=True, drop=True)
        season_result.reset_index(inplace=True)
        season_result['index'] += 1
        season_result.columns = ['Rank', 'Team', 'W', 'D', 'L', 'Pts', 'Score','Loss', 'GD']
        return season_result
    
    # Season, HomeTeam, AwayTeam 정보 등록, Last Season Rank(LSR)도 업데이트. 지난 시즌 순위의 차이를 의미(Away-Home)
    def Season_ht_at_LSR_update(self, epl_data, season, season_idx, ht, at, new_data={}): # input은 현 시즌 기준.
        season_data = DataTransformer.dhd.season_split(epl_data)[season_idx-1]
        last_sr = self.season_result(season_data=season_data)    # 지난 Season의 순위 Table.
        new_data['Season'] = season; new_data['HomeTeam'] = ht; new_data['AwayTeam'] = at
        
        if ht not in last_sr['Team'].values.tolist():
            htr = 21
        else:
            htr = last_sr[last_sr['Team']==ht].iloc[0, 0]
        
        if at not in last_sr['Team'].values.tolist():
            atr = 21
        else:
            atr = last_sr[last_sr['Team']==at].iloc[0, 0]
        
        new_data['LSR'] = atr - htr
        return new_data
    
    # .iloc을 활용하여 이전 n 경기들을 가져옴.
    # 홈팀/어웨이 팀의 지난 n경기 스탯으로부터 가져올 수 있는 정보: FTG, FTR, HTG, HTR, S, ST, C, F, Y, Rn, R1
    def last_n_stat_update(self, epl_data, ht, at, dt, last_game=5, new_data={}):
        home_team_last_games = self.dhd.all_team_matches(epl_data, ht)
        away_team_last_games = self.dhd.all_team_matches(epl_data, at)
        
        match_idx = home_team_last_games.index[home_team_last_games['DateTime'] == dt].tolist()[0]    #  현 경기의'DateTime' 정보를 추출
        if match_idx - last_game < 0:    # 홈팀의 지난 n경기가 실제 경기수를 초과할 경우 데이터가 있는 첫 경기부터 반영
            ht_last_n = home_team_last_games.iloc[0:match_idx]
        else:
            ht_last_n = home_team_last_games.iloc[match_idx-last_game:match_idx]
            
        if match_idx - last_game < 0:    # 어웨이팀의 지난 n경기가 실제 경기수를 초과할 경우 데이터가 있는 첫 경기부터 반영
            at_last_n = away_team_last_games.iloc[0:match_idx]
        else:
            at_last_n = away_team_last_games.iloc[match_idx-last_game:match_idx]
        
        ht_last_n_stats = [0]*11    # 순서대로 FTG, FTR, HTG, HTR, S, ST, C, F, Y, Rn, R1를 뜻하기로 한다.
        for idx, match in ht_last_n.iterrows():
            if idx == 0:
                if match['HomeTeam'] == ht:
                    ht_last_n_stats[-1] += match['HR']
                else:
                    ht_last_n_stats[-1] += match['AR']
            
            if match['HomeTeam'] == ht:
                for n, stat in enumerate(match.loc[['FTHG', 'FTR', 'HTHG', 'HTR', 'HS', 'HST', 'HC', 'HF', 'HY', 'HR']].tolist()):
                    ht_last_n_stats[n] += stat
            else:
                temp_lst = match.loc[['FTAG', 'FTR', 'HTAG', 'HTR', 'AS', 'AST', 'AC', 'AF', 'AY', 'AR']].tolist()
                if temp_lst[1] != 1:
                    temp_lst[1] = 3-temp_lst[1]
                if temp_lst[3] != 1:
                    temp_lst[3] = 3-temp_lst[3]
                for n, stat in enumerate(temp_lst):
                    ht_last_n_stats[n] += stat
        
        at_last_n_stats = [0]*11    # 순서대로 FTG, FTR, HTG, HTR, S, ST, C, F, Y, Rn, R1를 뜻하기로 한다.
        for idx, match in at_last_n.iterrows():
            if idx == 0:
                if match['HomeTeam'] == at:
                    at_last_n_stats[-1] += match['HR']
                else:
                    at_last_n_stats[-1] += match['AR']
            
            if match['HomeTeam'] == at:
                for n, stat in enumerate(match.loc[['FTHG', 'FTR', 'HTHG', 'HTR', 'HS', 'HST', 'HC', 'HF', 'HY', 'HR']].tolist()):
                    at_last_n_stats[n] += stat
            else:
                temp_lst = match.loc[['FTAG', 'FTR', 'HTAG', 'HTR', 'AS', 'AST', 'AC', 'AF', 'AY', 'AR']].tolist()
                if temp_lst[1] != 1:
                    temp_lst[1] = 3-temp_lst[1]
                if temp_lst[3] != 1:
                    temp_lst[3] = 3-temp_lst[3]
                for n, stat in enumerate(temp_lst):
                    at_last_n_stats[n] += stat
        
        key_list = ['FTG', 'FTR', 'HTG', 'HTR', 'S', 'ST', 'C', 'F', 'Y', 'Rn', 'R1']
        for i, key in enumerate(key_list):
            new_data[key] = ht_last_n_stats[i] - at_last_n_stats[i]
        return new_data
    
    # CSR(Current Season Rank) 업데이트. 현 시즌 순위의 차이를 의미(Away-Home), ignore값이 있으면 설정한 만큼은 default 0으로 함.
    def CSR_update(self, epl_data, season_idx, idx, ht, at, ignore=3, new_data={}):
        season_data = self.dhd.season_split(epl_data, index_reset=False)[season_idx]
        current_data = epl_data.iloc[season_idx*380:idx, :]
        current_result = self.season_result(current_data)
        if len(current_data) <= ignore*10:
            new_data['CSR'] = 0
        else:
            new_data['CSR'] = current_result[current_result['Team']==at].iloc[0, 0] - current_result[current_result['Team']==ht].iloc[0, 0]
        return new_data
    
    # LHAP(Last Home-Away Points) 업데이트.
    def LHAP_update(self, epl_data, ht, at, dt, last_game=3, new_data={}):
        ht_matches = self.dhd.home_matches(epl_data, ht)
        at_matches = self.dhd.away_matches(epl_data, at)
        
        match_idx = ht_matches.index[ht_matches['DateTime'] == dt].tolist()[0]    #  현 경기의'DateTime' 정보를 추출
        if match_idx - last_game < 0:    # 홈팀의 지난 n경기가 실제 경기수를 초과할 경우 데이터가 있는 첫 경기부터 반영
            ht_last_n = ht_matches.iloc[0:match_idx]
        else:
            ht_last_n = ht_matches.iloc[match_idx-last_game:match_idx]
            
        if match_idx - last_game < 0:    # 어웨이팀의 지난 n경기가 실제 경기수를 초과할 경우 데이터가 있는 첫 경기부터 반영
            at_last_n = at_matches.iloc[0:match_idx]
        else:
            at_last_n = at_matches.iloc[match_idx-last_game:match_idx]
        
        ht_pts = ht_last_n['FTR'].sum()
        at_pts = 0
        for idx, match in at_last_n.iterrows():
            if match['FTR'] == 1:
                at_pts += 1
            else:
                at_pts += (3-match['FTR'])
        new_data['LHAP'] = ht_pts - at_pts
        return ht_last_n, at_last_n, new_data
    
    # LMP(Last Match Pts), LMHAP(Last Match Hmoe-Away Pts) 업데이트.
    def LMP_LMHAP_update(self, epl_data, ht, at, dt, last_game_LMP=5, last_game_LMHAP=5, new_data={}):
        all_matches = self.dhd.all_two_team_matches(epl_data, ht, at, home_setting=False)
        home_matches = self.dhd.all_two_team_matches(epl_data, ht, at, home_setting=True)
        
        all_match_idx = all_matches.index[all_matches['DateTime'] == dt].tolist()[0]    #  현 경기의'DateTime' 정보를 추출
        home_match_idx = home_matches.index[home_matches['DateTime'] == dt].tolist()[0]    #  현 경기의'DateTime' 정보를 추출
        
        if all_match_idx - last_game_LMP < 0:    # 홈팀의 지난 n경기가 실제 경기수를 초과할 경우 데이터가 있는 첫 경기부터 반영
            all_last_n = all_matches.iloc[0:all_match_idx]
        else:
            all_last_n = all_matches.iloc[all_match_idx-last_game_LMP:all_match_idx]
            
        if home_match_idx - last_game_LMHAP < 0:    # 어웨이팀의 지난 n경기가 실제 경기수를 초과할 경우 데이터가 있는 첫 경기부터 반영
            home_last_n = home_matches.iloc[0:home_match_idx]
        else:
            home_last_n = home_matches.iloc[home_match_idx-last_game_LMHAP:home_match_idx]
        
        # LMP 먼저
        home_pts = 0; away_pts = 0
        for idx, match in all_last_n.iterrows():
            if match['FTR'] == 1:
                continue
            else:
                hp = match['FTR']; ap = 3-match['FTR']
                if match['HomeTeam'] == ht:
                    home_pts += hp; away_pts += ap
                else:
                    home_pts += ap; away_pts += hp
        new_data['LMP'] = home_pts - away_pts
        
        # LMHAP
        home_pts = 0; away_pts = 0
        for idx, match in home_last_n.iterrows():
            if match['FTR'] == 1:
                continue
            else:
                home_pts += match['FTR']; away_pts += 3-match['FTR']
        new_data['LMHAP'] = home_pts - away_pts
        return new_data

In [5]:
# 머신러닝에 적합하도록 원본 데이터를 변환. 첫 시즌의 데이터(00-01시즌)는 지난 시즌 순위 참고용으로만 사용하게 된다.
# last_game: 지난 n 경기의 승/무/패 결과를 반영하여 적용, average_stat: 지난 n 경기의 스탯을 반영하여 적용, ignore: 시즌 초기 n경기 순위 결과 무시
# rewrite_csv가 True면 처음부터. 단, csv_file이 'w' 모드일 때에만 가능.
# csv_file이 'r'모드이면 그대로 가져와서 반환. 'w'모드이면 파일을 작성.
# write_unit은 w 모드일 때 끊어서 작성하는 개수.
def data_transform(epl_data, last_game_LHAP = 5, last_game_LMP = 5, last_game_LMHAP = 5, average_stat = 5, ignore = 3, csv_file = 'r', csv_file_name = None, write_unit = False, rewrite_csv = False):
    if csv_file == 'r':
        if rewrite_csv == True:
            raise ValueError('csv_file 파라미터 오류: 쓰기(w)모드일 때에만 rewrite_csv를 True로 입력할 수 있음')
        if write_unit != False:
            raise ValueError('csv_file 파라미터 오류: 쓰기(w)모드일 때에만 write_unit을 입력할 수 있음')
        epl_data_trans = pd.read_csv(csv_file_name, encoding='cp949')
        return epl_data_trans
    
    elif csv_file != 'w':
        raise ValueError('csv_file 파라미터 오류: 쓰기(w)모드 혹은 읽기(r)모드 중에 선택')
    
    # csv_file 모드가 'w'모드인 경우.
    if type(write_unit) != int:
        raise ValueError('write_unit 파라미터 오류: 숫자나 False 값만 입력할 수 있음')
    if write_unit <= 0:
        raise ValueError('write_unit 파라미터 오류: 0보다 작을 수 없음')
    
    if rewrite_csv == True:
        col_names = ['Season', 'HomeTeam', 'AwayTeam', 'FTG', 'FTR', 'HTG', 'HTR', 'S', 'ST', 'C', 'F', 'Y', 'Rn', 'R1', 'LSR', 'CSR', 'LHAP', 'LMP', 'LMHAP', 'Result']
        epl_data_trans = pd.DataFrame(data=[], columns=col_names)
        start_point = 0
    else:
        col_names = ['Season', 'HomeTeam', 'AwayTeam', 'FTG', 'FTR', 'HTG', 'HTR', 'S', 'ST', 'C', 'F', 'Y', 'Rn', 'R1', 'LSR', 'CSR', 'LHAP', 'LMP', 'LMHAP', 'Result']
        epl_data_trans = pd.read_csv(csv_file_name, encoding='cp949')
        start_point = 380 + len(epl_data_trans)
    
    dtf = DataTransformer()    # 데이터 변환 유틸리티 객체
    
    for idx, match in epl_data.iterrows():
        if idx < 380:    # 첫 시즌인 00-01시즌은 데이터로 사용하지 않는다.
            continue
        
        if idx < start_point:    # 이미 쓴 데이터는 지나친다.
            continue
        
        if idx >= write_unit:    # 단위를 끊어서 작성하기 위함.
            break
        
        new_data = {}
        for column in col_names:
            new_data[column] = None    # Column들을 Key로 등록
        
        season = match['Season']; ht = match['HomeTeam']; at = match['AwayTeam']; dt = match['DateTime']    # 여기서는 그냥 가져와도 문제x(loc, iloc 사용 안해도 됨)
        season_idx = int(season[2:4])
        
        # Season, HomeTeam, AwayTeam 정보 등록, Last Season Rank(LSR)도 업데이트. 지난 시즌 순위의 차이를 의미(Away-Home)
        dtf.Season_ht_at_LSR_update(epl_data, season, season_idx, ht, at, new_data)
        
        # Result 업데이트.
        new_data['Result'] = match['FTR']
        
        # .iloc을 활용하여 이전 n 경기들을 가져옴.
        # 홈팀/어웨이 팀의 지난 n경기 스탯으로부터 가져올 수 있는 정보: FTG, FTR, HTG, HTR, S, ST, C, F, Y, Rn, R1
        dtf.last_n_stat_update(epl_data, ht, at, dt, average_stat, new_data)
        
        # CSR(Current Season Rank) 업데이트. 현 시즌 순위의 차이를 의미(Away-Home), ignore값이 있으면 설정한 만큼은 0으로 함.
        dtf.CSR_update(epl_data, season_idx, idx, ht, at, ignore, new_data)
        
        # LHAP(Last Home-Away Points) 업데이트.
        dtf.LHAP_update(epl_data, ht, at, dt, last_game_LHAP, new_data)
        
        # LMP(Last Match Pts), LMHAP(Last Match Hmoe-Away Pts) 업데이트.
        dtf.LMP_LMHAP_update(epl_data, ht, at, dt, last_game_LMP, last_game_LMHAP, new_data)
        
        # 데이터를 새로운 DataFrame 객체에 추가
        epl_data_trans = epl_data_trans.append(new_data, ignore_index=True)    
        
        """"""
        print('파일 작성 진행상황: {0}'.format(idx))
    pd.DataFrame(columns=epl_data_trans.columns).to_csv(csv_file_name, index=False)
    epl_data_trans.to_csv(csv_file_name, header=None, mode='a')
    return 0

In [6]:
data_transform(epl_data, csv_file = 'w', csv_file_name = 'transformed_data.csv', write_unit = 12000, rewrite_csv = False)

파일 작성 진행상황: 8128
파일 작성 진행상황: 8129
파일 작성 진행상황: 8130
파일 작성 진행상황: 8131
파일 작성 진행상황: 8132
파일 작성 진행상황: 8133
파일 작성 진행상황: 8134
파일 작성 진행상황: 8135
파일 작성 진행상황: 8136
파일 작성 진행상황: 8137


0

In [6]:
ml_data = pd.read_csv('transformed_data.csv')
ml_data

,Season,HomeTeam,AwayTeam,FTG,FTR,HTG,HTR,S,ST,C,F,Y,Rn,R1,LSR,CSR,LHAP,LMP,LMHAP,Result
0,2001-02,Charlton,Everton,0,3,1.0,5,-4.0,2.0,7.0,-26.0,-9.0,-1.0,0.0,7,0,4,0,3,0
1,2001-02,Derby,Blackburn,-8,-3,0.0,0,-16.0,-18.0,-8.0,-1.0,-3.0,1.0,0.0,4,0,1,0,0,3
2,2001-02,Leeds,Southampton,8,5,5.0,3,15.0,18.0,11.0,7.0,4.0,1.0,0.0,6,0,11,0,3,3
3,2001-02,Leicester,Bolton,-1,-1,-2.0,2,7.0,7.0,-5.0,17.0,9.0,0.0,0.0,8,0,3,0,0,0
4,2001-02,Liverpool,West Ham,8,10,1.0,6,1.0,3.0,-7.0,-9.0,-7.0,0.0,0.0,12,0,9,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7753,2021-22,Chelsea,Leeds,9,8,6.0,11,92.0,28.0,43.0,53.0,10.0,0.0,0.0,5,12,8,9,12,3
7754,2021-22,Norwich,Man United,-5,-5,-2.0,-5,-23.0,-24.0,-13.0,-9.0,4.0,0.0,0.0,-19,-14,-8,-9,-9,0
7755,2021-22,Leicester,Newcastle,2,1,3.0,0,3.0,-11.0,5.0,12.0,-2.0,-2.0,0.0,7,8,-1,3,-3,N
7756,2021-22,Burnley,West Ham,-6,-2,-3.0,-5,-19.0,-19.0,-1.0,-20.0,0.0,-1.0,0.0,-11,-14,4,3,0,N


In [7]:
ml_data = pd.read_csv('transformed_data.csv', encoding='cp949')
ml_data.drop(['Season', 'HomeTeam', 'AwayTeam'], inplace = True, axis=1)
ml_data['Result'] = ml_data['Result'].replace({'3':1, '1':0, '0':-1, 'N':-2})
ml_data

,FTG,FTR,HTG,HTR,S,ST,C,F,Y,Rn,R1,LSR,CSR,LHAP,LMP,LMHAP,Result
0,0,3,1.0,5,-4.0,2.0,7.0,-26.0,-9.0,-1.0,0.0,7,0,4,0,3,-1
1,-8,-3,0.0,0,-16.0,-18.0,-8.0,-1.0,-3.0,1.0,0.0,4,0,1,0,0,1
2,8,5,5.0,3,15.0,18.0,11.0,7.0,4.0,1.0,0.0,6,0,11,0,3,1
3,-1,-1,-2.0,2,7.0,7.0,-5.0,17.0,9.0,0.0,0.0,8,0,3,0,0,-1
4,8,10,1.0,6,1.0,3.0,-7.0,-9.0,-7.0,0.0,0.0,12,0,9,3,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7753,9,8,6.0,11,92.0,28.0,43.0,53.0,10.0,0.0,0.0,5,12,8,9,12,1
7754,-5,-5,-2.0,-5,-23.0,-24.0,-13.0,-9.0,4.0,0.0,0.0,-19,-14,-8,-9,-9,-1
7755,2,1,3.0,0,3.0,-11.0,5.0,12.0,-2.0,-2.0,0.0,7,8,-1,3,-3,-2
7756,-6,-2,-3.0,-5,-19.0,-19.0,-1.0,-20.0,0.0,-1.0,0.0,-11,-14,4,3,0,-2


In [8]:
from sklearn.preprocessing import StandardScaler

# StandardScaler 객체 생성
scaler = StandardScaler()
#StandardScaler로 데이터 세트 변환. fit()과 transform() 호출.
scaler.fit(ml_data)
ml_data_scaled = scaler.transform(ml_data)

#transform() 시 스케일 변환된 데이터 세트가 NumPy ndarray로 반환돼 이를 DataFrame으로 변환
ml_data_df_scaled = pd.DataFrame(data=ml_data_scaled, columns=ml_data.columns)

ml_data_df_scaled.describe()

,FTG,FTR,HTG,HTR,S,ST,C,F,Y,Rn,R1,LSR,CSR,LHAP,LMP,LMHAP,Result
count,7.758000e+03,7.758000e+03,7758.000000,7.758000e+03,7758.000000,7.758000e+03,7.758000e+03,7.758000e+03,7758.000000,7.758000e+03,7758.000000,7.758000e+03,7.758000e+03,7.758000e+03,7.758000e+03,7.758000e+03,7.758000e+03
mean,4.396243e-17,7.327071e-17,0.000000,1.465414e-17,0.000000,-7.327071e-18,-4.396243e-17,-1.465414e-17,0.000000,-5.312127e-17,0.000000,-2.747652e-18,-1.259340e-17,-1.099061e-17,-1.190649e-17,-8.059778e-17,-4.762596e-17
std,1.000064e+00,1.000064e+00,1.000064,1.000064e+00,1.000064,1.000064e+00,1.000064e+00,1.000064e+00,1.000064,1.000064e+00,1.000064,1.000064e+00,1.000064e+00,1.000064e+00,1.000064e+00,1.000064e+00,1.000064e+00
min,-3.292331e+00,-2.917114e+00,-3.654189,-2.717489e+00,-2.485593,-2.926611e+00,-2.930551e+00,-2.594972e+00,-3.385340,-4.821032e+00,-25.441660,-2.214616e+00,-2.421394e+00,-3.556550e+00,-2.340262e+00,-3.018004e+00,-2.545309e+00
25%,-5.984229e-01,-7.299127e-01,-0.516557,-6.300931e-01,-0.670614,-6.819655e-01,-6.975842e-01,-6.920644e-01,-0.764155,-7.885294e-02,-0.013111,-7.752918e-01,-7.489822e-01,-6.643569e-01,-4.434560e-01,-3.078370e-01,-1.370062e+00
50%,-2.115699e-02,-8.457853e-04,-0.167932,-3.874516e-03,-0.239941,-9.126934e-02,-1.721802e-01,-2.648810e-01,0.042364,-7.885294e-02,-0.013111,-2.711559e-04,2.290044e-02,1.068946e-01,3.074547e-02,-3.078370e-01,-1.948142e-01
75%,7.485309e-01,7.282212e-01,0.529320,6.223441e-01,0.459903,6.766357e-01,6.159258e-01,4.341465e-01,0.647252,-7.885294e-02,-0.013111,7.747495e-01,6.661359e-01,6.853331e-01,5.049469e-01,7.762298e-01,9.804334e-01
max,4.789392e+00,2.550889e+00,3.666951,2.918479e+00,3.420780,4.102674e+00,3.571323e+00,3.657440e+00,3.470067,3.477781e+00,25.415439,2.214074e+00,2.467195e+00,2.227836e+00,2.401753e+00,2.402330e+00,9.804334e-01


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

In [10]:
# X, y 데이터 분할 및 머신러닝 데이터 분할

X_ml_data = ml_data_df_scaled.iloc[:, 0:-1]
y_ml_data = ml_data['Result']
# X_train, X_test, y_train, y_test = train_test_split(X_ml_data, y_ml_data, test_size=0.2, stratify=y_ml_data)

X_train = X_ml_data.iloc[:-3, :]
X_test = X_ml_data.iloc[-3:, :]
y_train = y_ml_data[:-3]
y_test = y_ml_data[-3:]

In [11]:
from sklearn.ensemble import VotingClassifier

dt_clf_list = []
for i in range(3):
    clf_name = 'LR'+str(i+1)
    clf = LogisticRegression(C=100000,solver='newton-cg')
    dt_clf_list.append((clf_name, clf))
for i in range(3):
    clf_name = 'GB'+str(i+1)
    clf = GradientBoostingClassifier()
    dt_clf_list.append((clf_name, clf))

vo_clf = VotingClassifier(estimators=dt_clf_list, voting='soft')

# VotingClassifier 학습/예측/평가
vo_clf.fit(X_train, y_train)
#pred = vo_clf.predict(X_test)
#print(pd.Series(pred).value_counts())
#print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

VotingClassifier(estimators=[('LR1',
                              LogisticRegression(C=100000, solver='newton-cg')),
                             ('LR2',
                              LogisticRegression(C=100000, solver='newton-cg')),
                             ('LR3',
                              LogisticRegression(C=100000, solver='newton-cg')),
                             ('GB1', GradientBoostingClassifier()),
                             ('GB2', GradientBoostingClassifier()),
                             ('GB3', GradientBoostingClassifier())],
                 voting='soft')

In [12]:
def custom_threshold(n, pred_proba):
    prediction_array = np.array([])
    for lst in pred_proba:
        lst[1] *= n
        mx = lst.max()
        prediction_array = np.append(prediction_array, lst.tolist().index(mx)-1)
    prediction_array.astype('int64')
    return prediction_array

In [13]:
def recall_precision_draw_rate(pred, y_test):
    pred_series = pd.Series(pred)
    answer_series = pd.Series(y_test)
    answer_series = answer_series.reset_index(drop=True)

    pred_answer_df = pd.concat([pred_series, answer_series], axis=1)
    pred_answer_df.rename(columns = {0 : 'prediction', 'Result':'answer'}, inplace = True)

    # 실제 무승부 중에서 무승부 예측의 확률
    answer_len = len(pred_answer_df[pred_answer_df['answer'] == 0])
    draw_len = len(pred_answer_df[(pred_answer_df['answer'] == 0)&(pred_answer_df['prediction'] == 0)])
    try:
        recall = draw_len/answer_len
    except ZeroDivisionError:
        recall = 0

    # 무승부 예측 중에서 실제 무승부인 확률
    answer_pre_len = len(pred_answer_df[pred_answer_df['prediction'] == 0])
    draw_pre_len = len(pred_answer_df[(pred_answer_df['answer'] == 0)&(pred_answer_df['prediction'] == 0)])
    try:
        precision = draw_pre_len/answer_pre_len
    except ZeroDivisionError:
        precision = 0
    return recall, precision

In [14]:
pred = vo_clf.predict(X_test)

In [15]:
# Threshold 지수를 1.5로 높였을 때
pred_customed = custom_threshold(1.465, vo_clf.predict_proba(X_test))
accuracy = accuracy_score(pred_customed, y_test)
recall, precision = recall_precision_draw_rate(pred_customed, y_test)
print('정확도:{0:.2f}%, 재현율:{1:.2f}%, 정밀도:{2:.2f}%'.format(accuracy*100, recall*100, precision*100))
print(pd.Series(pred_customed).value_counts())

정확도:0.00%, 재현율:0.00%, 정밀도:0.00%
 0.0    1
-1.0    1
 1.0    1
dtype: int64


In [17]:
prediction = pd.Series(pred_customed).astype("int64")
result = pd.Series(y_test)
result.reset_index(inplace=True, drop= True)
ml_data = pd.read_csv('transformed_data.csv')

ml_recent_prediction = ml_data[['HomeTeam','AwayTeam']].iloc[-3:]
ml_recent_prediction.reset_index(drop=True, inplace=True)

comparison_df = pd.concat([ml_recent_prediction, prediction], axis=1)
comparison_df.rename(columns = {0: "prediction"}, inplace=True)
comparison_df = comparison_df.replace({1:'H', 0:'D', -1:'A'})
comparison_df

,HomeTeam,AwayTeam,prediction
0,Leicester,Newcastle,H
1,Burnley,West Ham,A
2,Crystal Palace,Everton,D
